<a href="https://colab.research.google.com/github/halfbug/colab/blob/main/retriever-vector-db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **LangChain and OpenAI installation**


In [125]:
try:
  import openai
  from langchain_openai import ChatOpenAI
except:
    !pip install openai
    !pip install langchain
    !pip install langchain-openai

In [126]:
import openai
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# llm = ChatOpenAI(openai_api_key=os.getenv('OPENAI_API_KEY'))
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [197]:
from langchain.chat_models import ChatOpenAI
import datetime
current_date = datetime.datetime.now().date()
# llm_name = "gpt-4"
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)
llm = ChatOpenAI(model_name=llm_name, temperature=0)
llm.invoke("Hello world!")

gpt-3.5-turbo


AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop', 'logprobs': None})

# Scraper

In [128]:
import re
import requests
from bs4 import BeautifulSoup

def extract_availability(url):
    # Function to scrape the webpage for availability information
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
         # Search for specific keywords like "stock availability" on the webpage

        body_content = soup.find('body')  # Find the body tag

        if body_content:
           # Extract only the text content from the body tag
            text_content = body_content.get_text(strip=True)
            # Search for either "stock" or "stock availability" in the text content
            # print(text_content)
            #  availability_pattern = re.compile(r"(?:stock|stock|availability|Number\sof\sItems\s*(\d+)|in\s*stock|(\w+)\s*stock\s*(\w+))", re.IGNORECASE)

            availability_pattern = re.compile(r"(?:stock|stock|availability|in\s*stock)", re.IGNORECASE)
            match = re.search(availability_pattern, text_content)
            # print(match.groups())
            if match:
                # print(match.group(0))
                return f"Availability: {match.group(0).strip()}"
        # If specific keywords not found, return "Scraped Availability: Unknown"
        return "Availability: Unknown"
    except Exception as e:
        print(f"Error occurred while scraping: {e}")
        return "Availability: Unknown"

In [129]:
def scrape_text(url):
    # Function to scrape the webpage for availability information
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
         # Search for specific keywords like "stock availability" on the webpage

         # Step 3: Extract metadata
        meta_tags = soup.find_all('meta')
        metadata = {}
        for tag in meta_tags:
            if 'name' in tag.attrs:
                name = tag.attrs['name']
                content = tag.attrs.get('content', '')
                metadata[name] = content
            elif 'property' in tag.attrs:  # For OpenGraph metadata
                property = tag.attrs['property']
                content = tag.attrs.get('content', '')
                metadata[property] = content

        # Display the metadata
        # for key, value in metadata.items():
        #     print(f"{key}: {value}")

        # print( metadata)

        meta_tag = extract_product_info([metadata])

            # Find all link elements with the 'as' attribute set to 'image'
        image_links = soup.find_all('link', {'rel': 'preload', 'as': 'image'})

        # Extract and print the href attributes
        images = ''
        for link in image_links:
            image_url = link.get('href')
            if image_url:
                if image_url.startswith("//"):
                  # Add protocol (http assumed):
                  image_url= f"http:{image_url}"
                images = images +","+ image_url
                # print(image_url)

        body_content = soup.find('body')  # Find the body tag

        if body_content:
           # Extract only the text content from the body tag
            text_content = body_content.get_text(strip=True)
            # Search for either "stock" or "stock availability" in the text content
            # print(text_content)
            string_encode = text_content.encode("ascii", "ignore")
            text_content = string_encode.decode()
            return text_content + meta_tag + "Images : " + images
        return f"Content: Unknown on {url}"
    except Exception as e:
        print(f"Error occurred while scraping: {e}")
        return f"Content: Unknown on {url}"

In [203]:
def format_string(input_string):
    formatted_string = ""

    # Split the input string into separate lines
    lines = input_string.split('\n')

    # Iterate through each line
    for line in lines:
        # Check if the line contains a colon ':'
        if ':' in line:
            # Split the line into key and value based on the colon ':'
            key, value = line.split(':', 1)
            # Format the key and value and add them to the formatted string
            formatted_string += f"{key.strip()}: {value.strip()} "
        else:
            # If the line does not contain a colon, add it as is to the formatted string
            formatted_string += f"{line.strip()} "

    return formatted_string

# Example usage
# input_string = "($)Date of the price:010203040506070809101112/01020304050607080910111213141516171819202122232425262728293031/Pleasesign into provide feedback.Submit FeedbackProduct detailsProduct Dimensions\n                                    \n                                        :\n                                    6 x 6 x 6 inches; 11.04 ouncesItem model number\n                                    \n                                        :', ':\n                                    CHAR-CAN-3WickDate First Available\n                                    \n                                        :\n                                    March 16, 2019Manufacturer\n                                    \n                                        :\n                                    APOTHEKEASIN\n                                    \n                                        :', 'B07PMY9PK6Customer Reviews:4.24.2 out of 5 stars147 ratingsExplore more recommendationsBrief content visible, double tap to read full content.Full content visible, double tap to read brief content.VideosPage1of1Start OverPage 1 of 1Previous pageVideos for this product1:14Click to play videoThe best gift all year long, a candle that lasts 120-150 hours!xAJohnsonVideos for this product1:08Click to play video#thisorthat aesthetic candle favoritesBonnies Favorite"
# formatted_output = format_string(input_string)
# print(formatted_output)


($)Date of the price: 010203040506070809101112/01020304050607080910111213141516171819202122232425262728293031/Pleasesign into provide feedback.Submit FeedbackProduct detailsProduct Dimensions  :  6 x 6 x 6 inches; 11.04 ouncesItem model number  : ', ': CHAR-CAN-3WickDate First Available  :  March 16, 2019Manufacturer  :  APOTHEKEASIN  : ', 'B07PMY9PK6Customer Reviews:4.24.2 out of 5 stars147 ratingsExplore more recommendationsBrief content visible, double tap to read full content.Full content visible, double tap to read brief content.VideosPage1of1Start OverPage 1 of 1Previous pageVideos for this product1:14Click to play videoThe best gift all year long, a candle that lasts 120-150 hours!xAJohnsonVideos for this product1:08Click to play video#thisorthat aesthetic candle favoritesBonnies Favorite 


In [204]:
def google_scraper(query) :
    try:
        response = requests.get("https://www.google.com/search?q="+query.lower().replace(' ', '+')+"upc")
        soup = BeautifulSoup(response.content, 'html.parser')

        # find first five link's content
        anchor_tags = soup.find_all('a', href=True)
        hrefs = []
        content = ""
        for tag in anchor_tags:
          href = tag.get('href')
          if href and href.startswith("/url?q=https") and "google" not in href.lower():  # Check if href exists (it might be None)
            parts = href.split('&', 1)
            parts = parts[0].split('/url?q=',1)
            # print(parts)
            hrefs.append(parts[1])
            scraped_content = scrape_text(parts[1])
            content += scraped_content + "\n ***"
            # print(scraped_content)
        # print(hrefs[:5])



        body_content = soup.find('body')  # Find the body tag

        if body_content:
           # Extract only the text content from the body tag
            text_content = body_content.get_text(strip=True)
            # print("\033[34m" + text_content )
            return format_string(content)
    except Exception as e:
        print(f"Error occurred while scraping: {e}")
        return "No search result found: Unknown"
google_scraper("Apotheke 3 wick candles collection")

'Skip to main contentShop by categoryShop by categoryEnter your search keywordAll CategoriesAdvancedHi (Sign into bid or buy)Daily DealsBrand OutletGift CardsHelp & ContactSellWatchlistExpand Watch ListLoading...Sign into see your user informationMy eBayExpand My eBaySummaryRecently ViewedBids/OffersWatchlistPurchase HistoryBuy AgainSellingSaved SearchesSaved SellersMy GarageSizesMessagesCollection betaThe eBay vaultNotificationExpand CartLoading...Something went wrong. View cart for details.{"delay": 300}Back to home page|Listed in category:breadcrumbHome & GardenCandles & Home FragranceCandlesShare|Add to WatchlistPicture 1 of 7Picture 1 of 7Have one to sell?Sell nowApotheke 3-Wick Candles Collection CharcoalDizon Group(9552)98.3% positiveSeller\'s other itemsSeller\'s other itemsContact sellerUS $109.95No Interest if paid in full in 6 mo on $99+ withPayPal Credit*Condition:NewNewQuantity:5 availableBuy It NowApotheke 3-Wick Candles Collection CharcoalSign in to check outCheck out as

In [131]:
import re

def extract_product_info(meta_tags):
    product_info = ""
    for meta_tag in meta_tags:
        if 'og:title' in meta_tag:
            product_info += f"\n ***Title: {meta_tag['og:title']}\n"
        if 'og:description' in meta_tag:
            product_info += f"Description: {meta_tag['og:description']}\n"
        if 'og:price:amount' in meta_tag and 'og:price:currency' in meta_tag:
            product_info += f"Price: {meta_tag['og:price:amount']} {meta_tag['og:price:currency']}\n"
        if 'og:image' in meta_tag:
                if len(meta_tag['og:image']) <= 200:  # Check if URL length is less than or equal to 200
                    product_info += f"Image URL: {meta_tag['og:image']} \n"
        if 'og:url' in meta_tag:
            product_info += f"URL: {meta_tag['og:url']} \n"
            # Call function to extract availability information
            availability_info = extract_availability(meta_tag['og:url'])
            product_info += f"{availability_info}\n"
            # Extract ASIN from URL if it's an Amazon link
            asin_match = re.search(r'/dp/(\w+)', meta_tag['og:url'])
            if asin_match:
                product_info += f"ASIN: {asin_match.group(1)} \n"
        if 'og:site_name' in meta_tag:
            product_info += f"Site Name: {meta_tag['og:site_name']} \n"
        if 'og:availability' in meta_tag:
            product_info += f"Availability: {meta_tag['og:availability']} \n"
        if 'og:type' in meta_tag:
                product_info += f"type: {meta_tag['og:type']} \n"
        else:
          product_info += f"type: text \n"
    return product_info

# Example usage:
# meta_tags = [{'apple-itunes-app': 'app-id=529356019, app-argument=https://www.crateandbarrel.com/apotheke-canvas-scented-3-wick-candle/s569504',
#  'og:image': 'https://cb.scene7.com/is/image/Crate/ApothekeCnvs3WickCandleSSF23', 'og:type': 'product', 'og:image:alt': 'Apotheke Canvas-Scented 3-Wick Candle',
#   'twitter:card': 'summary_large_image', 'twitter:title': 'Apotheke Canvas-Scented 3-Wick Candle + Reviews | Crate & Barrel', 'og:site_name': 'Crate & Barrel', 'og:availability':
#    'InStock', 'og:price:amount': '$78.00', 'author': 'Crate & Barrel', 'og:title': 'Apotheke Canvas-Scented 3-Wick Candle + Reviews | Crate & Barrel', 'og:price:currency': 'USD', 'language':
#     'English', 'og:description': 'Shop Apotheke Canvas-Scented 3-Wick Candle.  This hand-poured candle fills the room with clean, refreshing scent.  Infused with essences of crisp linen, white musk,
#      lily of the valley and dew drop, the soy-blend wax candle burns for many hours releasing a sweet, fresh fragrance.', 'og:price:standard_amount': '$78.00', 'twitter:site': '@CrateandBarrel',
#      'viewport': 'width=device-width, initial-scale=1.0', 'fb:admins': '1769841725', 'og:url': 'https://www.amazon.com/dp/B09RTPD6ZN/ref=tsm_1_fb_lk'}]
# product_info = extract_product_info(meta_tags)
# print(product_info)

# Google Search API

In [132]:
from googleapiclient.discovery import build

API_KEY = userdata.get('GOOGLE_SEARCH_API')
# os.environ["GOOGLE_CSE_ID"] = userdata.get('GOOGLE_CSE_ID')

service = build("customsearch", "v1", developerKey=API_KEY)
# clientProd = build("shopping", "v1", developerKey=API_KEY)

In [133]:
def search_google(query):
    """
    Performs a custom search  using the Google Custom Search API.

    Args:
        query (str): The search query to use.
        num_results (int, optional): The number of image search results to return (default: 3).

    Returns:
        str: A string containing information about the image search results. Each result includes the image URL and its corresponding link.
    """
    # user_query= extract_text_between_hashes(query)
    results = service.cse().list(
        q=query, cx=userdata.get('GOOGLE_CSE_ID'),  # Replace with your custom search engine ID
        # searchType="image",  # Specify image search
        num=3
    ).execute()

    search_results = []
    for item in results.get("items", []):
        for key, value in item.items():
          print("\033[32m" + f"{key}: {value}" + "\033[0m")
        image_url = item["pagemap"]["cse_image"][0]["src"]
        print(item["pagemap"]["metatags"])
        search_results.append({"link":item["link"] ,
                              "title": item["title"] ,
                              "snippets":item["snippet"],
                              "image": image_url,
                              "meta": extract_product_info(item["pagemap"]["metatags"])})
        # search_results += extract_product_info(item["pagemap"]["metatags"])
        # search_results += f"image2: {image_url} \n"
        # search_results += f"link: {item['link']}"

    len(search_results)
    return search_results

# Example usage:
image_results = search_google("Apotheke 3 wick candles collection price images")
# image_results
print(image_results)

kind: customsearch#result
title: Apotheke 3-Wick Candles Collection : Health ... - Amazon.com
htmlTitle: Apotheke 3-Wick Candles Collection : Health ... - Amazon.com
link: https://www.amazon.com/Scented-Candles-Home-Soy-Wax-Jar-Candle/dp/B07PMY9PK6
displayLink: www.amazon.com
snippet: Captivating Fragrance: Transform any living space with the unforgettable scent of Charcoal 3-Wick Candle from APOTHEKE. Our candles are hand poured and made ...
htmlSnippet: Captivating Fragrance: Transform any living space with the unforgettable scent of Charcoal <b>3</b>-<b>Wick Candle</b> from <b>APOTHEKE</b>. Our <b>candles</b> are hand poured and made&nbsp;...
formattedUrl: https://www.amazon.com/Scented-Candles-Home-Soy.../B07PMY9PK6
htmlFormattedUrl: https://www.amazon.com/Scented-<b>Candles</b>-Home-Soy.../B07PMY9PK6
pagemap: {'cse_thumbnail': [{'src': 'https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcSn_XK9lxk8YHeoZ-1cwFl6vAw52HIMW2FN2RbGuGT2KqCwycqvSOYtY-o', 'width': '238', 'height': '212'}

In [134]:
def combined_scraped_api_search(query):
  scraped_results = google_scraper(query)
  search_api_results = search_google(query)
  return f"""{scraped_results} \n {search_api_results} """


# RAG

In [135]:
column =  "Apotheke 3 wick candles collection"

In [136]:
google_scraper(column)

'Google×Please clickhereif you are not redirected within a few seconds.AllImagesShoppingNewsMapsVideosBooksSearch toolsAny timeAny timePast hourPast 24 hoursPast weekPast monthPast yearAll resultsAll resultsVerbatim3-Wick Candles | APOTHEKEapothekeco.com › collections › 3-wick-candlesMade with a premium soy wax blend and essential and perfume-grade fragrance oils, our luxury scented 3-wick candles are designed to provide a clean-burning\xa0...Charcoal 3-Wick Candle - Apothekeapothekeco.com › products › charcoal-3-wick-candleRating4.4(125)·$78.00·In stockThis 3-Wick Scented Candle is made with a premium soy wax blend and essential and perfume-grade fragrance oils in a chic matte-black glass vessel. The FragranceCanvas 3-Wick Candle - Apothekeapothekeco.com › products › canvas-3-wick-candleRating5.0(8)·$78.00·In stockThis 3-Wick Scented Candle is made with a premium soy wax blend and essential and perfume-grade fragrance oils in a luxe white-translucent glass vessel. The\xa0...APOTHEKE M

## Document Splitting

In [205]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 700,chunk_overlap = 150)


In [206]:
splits = text_splitter.split_text(google_scraper(column))
len(splits)

216

In [207]:
splits


['Skip to main contentShop by categoryShop by categoryEnter your search keywordAll CategoriesAdvancedHi (Sign into bid or buy)Daily DealsBrand OutletGift CardsHelp & ContactSellWatchlistExpand Watch ListLoading...Sign into see your user informationMy eBayExpand My eBaySummaryRecently ViewedBids/OffersWatchlistPurchase HistoryBuy AgainSellingSaved SearchesSaved SellersMy GarageSizesMessagesCollection betaThe eBay vaultNotificationExpand CartLoading...Something went wrong. View cart for details.{"delay": 300}Back to home page|Listed in category:breadcrumbHome & GardenCandles & Home FragranceCandlesShare|Add to WatchlistPicture 1 of 7Picture 1 of 7Have one to sell?Sell nowApotheke 3-Wick Candles',
 "& GardenCandles & Home FragranceCandlesShare|Add to WatchlistPicture 1 of 7Picture 1 of 7Have one to sell?Sell nowApotheke 3-Wick Candles Collection CharcoalDizon Group(9552)98.3% positiveSeller's other itemsSeller's other itemsContact sellerUS $109.95Condition:NewNewQuantity:5 availableBuy It

In [198]:
docs = text_splitter.split_documents(google_scraper(column))
len(docs)
docs

AttributeError: 'str' object has no attribute 'page_content'

## Embeddings and Vectorstores
Let's take our splits and embed them.

In [140]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [180]:
!pip install chromadb

from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma/'

import sys
sys.path.append('../..')




In [218]:
!rm -rf ./docs/chroma  # remove old database files if any

In [181]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [1]:
vectordb = Chroma.from_texts(splits, embedding=embedding)

NameError: name 'Chroma' is not defined

In [ ]:
print(vectordb._collection.count())

In [211]:
vectordb.similarity_search(column, k=5)

[Document(page_content='***Title: Apotheke - Bamboo 3 Wick Candle'),
 Document(page_content='***Title: Apotheke - Bamboo 3 Wick Candle'),
 Document(page_content='Google×Please clickhereif you are not redirected within a few seconds.AllShoppingImagesNewsMapsVideosBooksSearch toolsAny timeAny timePast hourPast 24 hoursPast weekPast monthPast yearAll resultsAll resultsVerbatimDid you mean:apotheke 3 wick candles collectionasin upcamazonSponsoredⓘShop apotheke 3 wick candles collection asinupc amazonApotheke 3‑Wick Candles (Canvas, ...$78.00Amazon.comApotheke 3‑Wick Candles (Sea ...$78.00Amazon.comApotheke 3‑Wick Candles (Santal ...$78.00Amazon.comAPOTHEKE Market Collection Luxury ...$78.00Amazon.comAPOTHEKE Market Collection Luxury ...$49.95Amazon.comApotheke 3‑Wick Candles (Orange ...$78.00Amazon.comApotheke 3‑Wick Candles (Earl'),
 Document(page_content='3-Wick Scented Candle is made with a premium soy wax blend and essential and perfume-grade fragrance oils in a glossy blue-translucent

In [212]:
vectordb.max_marginal_relevance_search(column+" upc",k=5)

[Document(page_content='Google×Please clickhereif you are not redirected within a few seconds.AllShoppingImagesNewsMapsVideosBooksSearch toolsAny timeAny timePast hourPast 24 hoursPast weekPast monthPast yearAll resultsAll resultsVerbatimDid you mean:apotheke 3 wick candles collectionasin upcamazonSponsoredⓘShop apotheke 3 wick candles collection asinupc amazonApotheke 3‑Wick Candles (Canvas, ...$78.00Amazon.comApotheke 3‑Wick Candles (Sea ...$78.00Amazon.comApotheke 3‑Wick Candles (Santal ...$78.00Amazon.comAPOTHEKE Market Collection Luxury ...$78.00Amazon.comAPOTHEKE Market Collection Luxury ...$49.95Amazon.comApotheke 3‑Wick Candles (Orange ...$78.00Amazon.comApotheke 3‑Wick Candles (Earl'),
 Document(page_content='MPN. CHAR-CAN-3Wick. UPC. 602573896181. Brand. APOTHEKE. Scent. Charcoal. Type. Does Not Apply. Category. Home & Garden > Candles & Home\xa0...Apotheke 3-Wick Candles Collection - Amazon.comwww.amazon.com › Scented-Candles-Home-Soy-Wax-Jar-CandleRating4.2(147)·$78.00·In s

In [214]:
retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 5})

In [216]:
retriever.get_relevant_documents("Apotheke 3-Wick Candles Collection charcoal amazon ")


[Document(page_content='***Title: Apotheke 3-Wick Candles Collection Charcoal  | eBay\nDescription: Apotheke 3-Wick Candles Collection. - Manufacturer \u200f : \u200e APOTHEKE. - Manufacturer: \u200eAPOTHEKE. - Scent: Charcoal. - Scent: \u200eCharcoal. Product Information. - Part Number: \u200eCHAR-CAN-3Wick. - Date First Available \u200f : \u200e March 16, 2019.\nImage URL: https://i.ebayimg.com/images/g/EesAAOSwtkJlbm4r/s-l400.jpg \nURL: https://www.ebay.com/itm/155928752194 \nAvailability: Unknown\nSite Name: eBay \ntype: ebay-objects:item'),
 Document(page_content='fragrance oils, our luxury scented 3-wick candles are designed to provide a clean-burning\xa0...Charcoal 3-Wick Candle - Apothekeapothekeco.com › products › charcoal-3-wick-candleRating4.4(125)·$78.00·In stockThis 3-Wick Scented Candle is made with a premium soy wax blend and essential and perfume-grade fragrance oils in a chic matte-black glass vessel. The FragranceCanvas 3-Wick Candle - Apothekeapothekeco.com › product

# LangChain Expression Language (LCEL)

In [149]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableMap

In [150]:
template = """
As a search analyst, your task is to find specific information online. We've provided context to assist:

{context}

Answer the following question based solely on the context:

Question: {question}

Provide the requested information in this format:

Brand name:
Product name:
Product image(s): [provide at least one link]
Color: [list colors, separated by commas]
Category: [identify the product's category on Amazon]
Description: [include a brief product description]
Sold on Amazon: true / false
Amazon price: [indicate the price range]
ASIN: [list ASIN codes, separated by commas]
UPC: [mention the UPC]


"""
prompt = ChatPromptTemplate.from_template(template)

In [151]:
model = llm
output_parser = StrOutputParser()



In [193]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),  #vectordb.max_marginal_relevance_search((x["question"]),k=5)     #google_scraper(x["question"]),     #combined_scraped_api_search(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [217]:
print(chain.invoke({"question": "Apotheke 3-Wick Candles Collection charcoal asin upc"}))

Brand name: Apotheke
Product name: Apotheke 3-Wick Candles Collection Charcoal
Product image(s): https://i.ebayimg.com/images/g/EesAAOSwtkJlbm4r/s-l400.jpg
Color: Charcoal
Category: Home & Garden > Candles & Home
Description: Hand poured candles made with a proprietary blend of essential and perfume-grade oils for a long-lasting fragrance. Each 26 oz candle burns for a long time.
Sold on Amazon: true
Amazon price: $78.00
ASIN: B07P5JZQZ3
UPC: 602573896181


In [154]:
from langchain.callbacks import get_openai_callback


In [176]:
with get_openai_callback() as cb:
    result = chain.invoke({"question": "Apotheke 3-Wick Candles Collection charcoal"})
    print(cb)

Tokens Used: 769
	Prompt Tokens: 625
	Completion Tokens: 144
Successful Requests: 1
Total Cost (USD): $0.0012255
